In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

sajidma23m013_inaturalist_dataset_path = kagglehub.dataset_download('sajidma23m013/inaturalist-dataset')

print('Data source import complete.')


In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login(key='53b259076c07d0811d73bf26bfef7437e04dbf66')

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ma23m013 (ma23m013-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.model_selection import train_test_split
from torchvision import models

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define function for pretraining a model
def pretrain_model(model_name, freeze_percent, freeze_all_except_last_layer):
    '''
    This function initializes a pre-trained model based on the model_name argument,
    freezes layers based on the freeze_percent, and optionally freezes all layers except
    the last fully connected layer.

    Parameters:
    - model_name (str): The name of the model to load (e.g., 'resnet', 'googlenet')
    - freeze_percent (float): Percentage of layers to freeze (e.g., 0.25 for 25%)
    - freeze_all_except_last_layer (str): If 'Yes', only the last layer is trainable. If 'No', freeze layers based on freeze_percent.

    Returns:
    - pretrained_model (torch.nn.Module): The model with modified layers and frozen parameters.
    '''

    # Load pre-trained model based on the provided model name
    if model_name == 'resnet':
        pretrained_model = models.resnet50(pretrained=True)
    elif model_name == 'googlenet':
        pretrained_model = models.googlenet(pretrained=True)

    # Modify the final fully connected layer to match the number of classes (10 in this case)
    nodes_fc = pretrained_model.fc.in_features  # Get the input features of the last fully connected layer
    pretrained_model.fc = nn.Linear(nodes_fc, 10)  # Change to 10 classes (modify if necessary)

    # Calculate the number of layers to freeze based on the freeze_percent
    total_layers = sum(1 for _ in pretrained_model.children())
    num_freeze = int(total_layers * freeze_percent)

    # Freeze layers based on the 'freeze_all_except_last_layer' flag
    if freeze_all_except_last_layer == 'Yes':
        for name, parameter in pretrained_model.named_parameters():
            if not name.startswith('fc'):  # Exclude the last layer named 'fc'
                parameter.requires_grad = False  # Freeze this layer

    if freeze_all_except_last_layer == 'No':
        count = 0
        # Freeze layers based on the 'freeze_percent' argument
        for name, child in pretrained_model.named_children():
            if count < num_freeze:
                for parameter in child.parameters():
                    parameter.requires_grad = False  # Freeze the layer
            else:
                for parameter in child.parameters():
                    parameter.requires_grad = True  # Make the layer trainable
            count += 1

    return pretrained_model

# Example usage
freeze_percent = 0.25  # Freeze 25% of layers
freeze_all_except_last_layer = 'No'
model_name = 'googlenet'  # Using GoogLeNet
model = pretrain_model(model_name, freeze_percent, freeze_all_except_last_layer)
print(model)  # Print the model architecture


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:00<00:00, 190MB/s]


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [ ]:
# Example usage
freeze_percent = 0.25  # Freeze 25% of layers
freeze_all_except_last_layer = 'No'
model_name = 'resnet'  # Using ResNet
model = pretrain_model(model_name, freeze_percent, freeze_all_except_last_layer)
print(model)  # Print the model architecture


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 213MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.model_selection import train_test_split

def load_dataset(data_directory, apply_augmentation):
    """
    Load and preprocess image dataset with optional data augmentation.

    Args:
        data_directory (str): Path to the dataset directory.
        apply_augmentation (str): 'Yes' to apply data augmentation, 'No' otherwise.

    Returns:
        train_loader (DataLoader): DataLoader for the training set.
        val_loader (DataLoader): DataLoader for the validation set.
    """

    # Define image transformation pipeline
    if apply_augmentation == 'Yes':
        transform_pipeline = transforms.Compose([
            transforms.RandomResizedCrop(224),                       # Randomly crop and resize to 224x224
            transforms.RandomHorizontalFlip(),                       # Random horizontal flip
            transforms.ColorJitter(brightness=0.2,
                                   contrast=0.2,
                                   saturation=0.2,
                                   hue=0.1),                         # Random brightness, contrast, etc.
            transforms.RandomRotation(20),                           # Random rotation up to 20 degrees
            transforms.ToTensor(),                                   # Convert image to tensor
            transforms.Normalize((0.5, 0.5, 0.5),
                                 (0.5, 0.5, 0.5))                    # Normalize to [-1, 1] range
        ])
    else:
        transform_pipeline = transforms.Compose([
            transforms.Resize((224, 224)),                           # Resize to 224x224
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5),
                                 (0.5, 0.5, 0.5))
        ])

    # Load dataset from directory
    dataset = ImageFolder(root=data_directory, transform=transform_pipeline)

    # Split indices for training and validation sets
    train_indices, val_indices = train_test_split(
        list(range(len(dataset))), test_size=0.2, random_state=42
    )

    # Create samplers for training and validation
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    # DataLoaders with batch size of 32
    train_loader = DataLoader(dataset, batch_size=32, sampler=train_sampler)
    val_loader = DataLoader(dataset, batch_size=32, sampler=val_sampler)

    return train_loader, val_loader


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

def train_model_on_data(model, train_loader):
    """
    Train the given model on training data for one epoch.

    Args:
        model (nn.Module): The neural network to train.
        train_loader (DataLoader): DataLoader for the training set.

    Returns:
        avg_loss (float): Average training loss over the epoch.
        train_accuracy (float): Training accuracy as a percentage.
    """

    # Define loss function and optimizer
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Set model to training mode
    model.train()

    # Initialize counters for tracking performance
    cumulative_loss = 0.0
    total_correct = 0
    total_samples = 0

    # Training loop over batches
    for batch_images, batch_labels in train_loader:
        # Move data to device (GPU if available)
        batch_images, batch_labels = batch_images.to(device), batch_labels.to(device)

        # Forward pass
        predictions = model(batch_images)
        loss = loss_function(predictions, batch_labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate loss
        cumulative_loss += loss.item()

        # Compute number of correct predictions
        _, predicted_classes = torch.max(predictions, 1)
        total_correct += (predicted_classes == batch_labels).sum().item()
        total_samples += batch_labels.size(0)

    # Compute average loss and accuracy
    avg_loss = cumulative_loss / len(train_loader)
    train_accuracy = 100 * total_correct / total_samples

    return avg_loss, train_accuracy


In [ ]:
import torch
import torch.nn as nn

def evaluate_model_on_validation(model, validation_loader):
    """
    Evaluate the model on the validation set.

    Args:
        model (nn.Module): The trained model.
        validation_loader (DataLoader): DataLoader for the validation set.

    Returns:
        avg_val_loss (float): Average loss on the validation set.
        validation_accuracy (float): Accuracy on the validation set as a percentage.
    """

    # Set model to evaluation mode (disables dropout, batchnorm updates, etc.)
    model.eval()

    # Define loss function
    loss_function = nn.CrossEntropyLoss()

    # Initialize counters for loss and accuracy
    cumulative_val_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    # Disable gradient computation during evaluation
    with torch.no_grad():
        for batch_images, batch_labels in validation_loader:
            # Move data to GPU if available
            batch_images, batch_labels = batch_images.to(device), batch_labels.to(device)

            # Forward pass
            outputs = model(batch_images)
            loss = loss_function(outputs, batch_labels)
            cumulative_val_loss += loss.item()

            # Calculate accuracy
            _, predicted_classes = torch.max(outputs, 1)
            correct_predictions += (predicted_classes == batch_labels).sum().item()
            total_samples += batch_labels.size(0)

    # Compute average loss and accuracy
    avg_val_loss = cumulative_val_loss / len(validation_loader)
    validation_accuracy = 100 * correct_predictions / total_samples

    return avg_val_loss, validation_accuracy


In [ ]:
# Function to train and validate the model
def train_and_validate_model(model, train_loader, val_loader, num_epochs):
    """
    Trains the given model and evaluates on the validation set for each epoch.

    Args:
        model (nn.Module): The neural network to train.
        train_loader (DataLoader): DataLoader for training data.
        val_loader (DataLoader): DataLoader for validation data.
        num_epochs (int): Number of training epochs.
    """

    # Define the loss function and optimizer
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        # Training phase
        avg_train_loss, train_accuracy = train_model_on_data(model, train_loader)
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%')

        # Log training metrics to wandb
        wandb.log({'Train Loss': avg_train_loss})
        wandb.log({'Train Accuracy': train_accuracy})

        # Validation phase
        avg_val_loss, val_accuracy = evaluate_model_on_validation(model, val_loader)
        print(f'Epoch {epoch+1}/{num_epochs}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%')

        # Log validation metrics to wandb
        wandb.log({'Validation Loss': avg_val_loss})
        wandb.log({'Validation Accuracy': val_accuracy})
        wandb.log({'Epoch': epoch + 1})

    print('Training complete!')


In [ ]:
# Sweep configuration dictionary
sweep_config = {
    'method': 'bayes',  # 'bayes' will use Bayesian optimization to find optimal hyperparameters, other options are 'grid' or 'random'

    'metric': {
        'name': 'Validation Accuracy',  # Metric to optimize during the sweep
        'goal': 'maximize'  # Goal is to maximize the validation accuracy
    },

    'parameters': {

        'freeze_percent': {
            'values': [0.0, 0.25, 0.5, 0.75]  # Different percentages of layers to freeze during fine-tuning
        },

        'model_name': {
            'values': ['resnet', 'googlenet']  # Pre-trained model options for fine-tuning
        },

        'freeze_all_except_last_layer': {
            'values': ['Yes', 'No']  # Option to freeze all layers except the last one or not
        },

        'epochs': {
            'value': 5  # Number of epochs for each training run
        }
    }
}

# Initialize the W&B sweep using the config
sweep_id = wandb.sweep(sweep_config, project="DA6401_A-2_Part-B")  # Initialize sweep in the project "DA6401_A-2_Part-B"


Create sweep with ID: jilrfsdr
Sweep URL: https://wandb.ai/ma23m013-iit-madras/DA6401_A-2_Part-B/sweeps/jilrfsdr


In [ ]:
def main():
    '''
    WandB calls the main function each time with a different combination of hyperparameters.
    We can retrieve the configuration values from `wandb.config` and use them in our model training.

    This function initializes the W&B run, loads the pre-trained model with specified hyperparameters,
    prepares the dataset, and starts the training process.
    '''

    # Initialize W&B run
    with wandb.init() as run:
        # Define the run name based on hyperparameters for better tracking
        run_name = "ep" + str(wandb.config.epochs) + "_fp-" + str(wandb.config.freeze_percent) + "_model-" + str(wandb.config.model_name)
        wandb.run.name = run_name  # Set the name for this run in W&B

        # Load the pre-trained model based on configuration
        model = pretrain_model(
            model_name=wandb.config.model_name,
            freeze_percent=wandb.config.freeze_percent,
            freeze_all_except_last_layer=wandb.config.freeze_all_except_last_layer
        )

        # Move the model to the appropriate device (CPU or GPU)
        model = model.to(device)

        # Define the directory where the training data is located
        data_dir = '/kaggle/input/inaturalist-dataset/inaturalist_12K/train'

        # Load the training and validation data with or without data augmentation
        train_data, val_data = load_dataset(data_dir, apply_augmentation='No')

        # Train the model with the loaded data and configurations
        train_and_validate_model(model, train_data, val_data, num_epochs=wandb.config.epochs)


# Start the W&B sweep, calling the main function for the specified number of times
wandb.agent(sweep_id, function=main, count=10)  # Number of runs = 1 in this case, but can be increased
wandb.finish()  # Finish the W&B run after the sweep is complete


wandb: Agent Starting Run: m92ku3du with config:
wandb: 	epochs: 5
wandb: 	freeze_all_except_last_layer: Yes
wandb: 	freeze_percent: 0
wandb: 	model_name: resnet
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch 1/5, Train Loss: 1.1326, Train Accuracy: 63.26%
Epoch 1/5, Validation Loss: 0.8604, Validation Accuracy: 72.95%
Epoch 2/5, Train Loss: 0.8336, Train Accuracy: 72.92%
Epoch 2/5, Validation Loss: 0.7892, Validation Accuracy: 73.95%
Epoch 3/5, Train Loss: 0.7901, Train Accuracy: 73.91%
Epoch 3/5, Validation Loss: 0.8360, Validation Accuracy: 73.00%
Epoch 4/5, Train Loss: 0.7579, Train Accuracy: 74.80%
Epoch 4/5, Validation Loss: 0.7916, Validation Accuracy: 74.10%
Epoch 5/5, Train Loss: 0.7246, Train Accuracy: 75.81%
Epoch 5/5, Validation Loss: 0.8096, Validation Accuracy: 73.75%
Training complete!


Epoch,▁▃▅▆█
Train Accuracy,▁▆▇▇█
Train Loss,█▃▂▂▁
Validation Accuracy,▁▇▁█▆
Validation Loss,█▁▆▁▃
Epoch,5
Train Accuracy,75.80948
Train Loss,0.72465
Validation Accuracy,73.75
Validation Loss,0.80959


wandb: Agent Starting Run: 5lrmqu76 with config:
wandb: 	epochs: 5
wandb: 	freeze_all_except_last_layer: Yes
wandb: 	freeze_percent: 0
wandb: 	model_name: googlenet


Epoch 1/5, Train Loss: 1.4553, Train Accuracy: 55.03%
Epoch 1/5, Validation Loss: 1.1061, Validation Accuracy: 65.15%
Epoch 2/5, Train Loss: 1.0713, Train Accuracy: 65.36%
Epoch 2/5, Validation Loss: 0.9998, Validation Accuracy: 67.55%
Epoch 3/5, Train Loss: 0.9985, Train Accuracy: 67.36%
Epoch 3/5, Validation Loss: 0.9973, Validation Accuracy: 68.25%
Epoch 4/5, Train Loss: 0.9669, Train Accuracy: 68.15%
Epoch 4/5, Validation Loss: 0.9684, Validation Accuracy: 67.55%
Epoch 5/5, Train Loss: 0.9437, Train Accuracy: 68.00%
Epoch 5/5, Validation Loss: 0.9623, Validation Accuracy: 67.50%
Training complete!


...

Epoch 1/5, Train Loss: 1.1337, Train Accuracy: 64.25%
Epoch 1/5, Validation Loss: 0.8534, Validation Accuracy: 72.75%
Epoch 2/5, Train Loss: 0.8432, Train Accuracy: 72.53%
Epoch 2/5, Validation Loss: 0.7935, Validation Accuracy: 73.65%
Epoch 3/5, Train Loss: 0.7837, Train Accuracy: 74.63%
Epoch 3/5, Validation Loss: 0.8190, Validation Accuracy: 72.70%
Epoch 4/5, Train Loss: 0.7764, Train Accuracy: 74.47%
Epoch 4/5, Validation Loss: 0.8951, Validation Accuracy: 71.50%
Epoch 5/5, Train Loss: 0.7310, Train Accuracy: 75.53%
Epoch 5/5, Validation Loss: 0.8354, Validation Accuracy: 73.90%
Training complete!


Epoch,▁▃▅▆█
Train Accuracy,▁▆▇▇█
Train Loss,█▃▂▂▁
Validation Accuracy,▅▇▄▁█
Validation Loss,▅▁▃█▄
Epoch,5
Train Accuracy,75.53444
Train Loss,0.73105
Validation Accuracy,73.9
Validation Loss,0.83539


wandb: Agent Starting Run: i9z07opa with config:
wandb: 	epochs: 5
wandb: 	freeze_all_except_last_layer: Yes
wandb: 	freeze_percent: 0.25
wandb: 	model_name: resnet


Epoch 1/5, Train Loss: 1.1304, Train Accuracy: 63.32%
Epoch 1/5, Validation Loss: 0.8743, Validation Accuracy: 71.10%
Epoch 2/5, Train Loss: 0.8297, Train Accuracy: 72.61%
Epoch 2/5, Validation Loss: 0.8203, Validation Accuracy: 72.20%
Epoch 3/5, Train Loss: 0.7800, Train Accuracy: 74.30%
Epoch 3/5, Validation Loss: 0.8960, Validation Accuracy: 70.90%
Epoch 4/5, Train Loss: 0.7620, Train Accuracy: 74.71%
Epoch 4/5, Validation Loss: 0.8130, Validation Accuracy: 74.00%
Epoch 5/5, Train Loss: 0.7287, Train Accuracy: 75.65%
Epoch 5/5, Validation Loss: 0.8214, Validation Accuracy: 74.15%
Training complete!


Epoch,▁▃▅▆█
Train Accuracy,▁▆▇▇█
Train Loss,█▃▂▂▁
Validation Accuracy,▁▄▁██
Validation Loss,▆▂█▁▂
Epoch,5
Train Accuracy,75.64696
Train Loss,0.72868
Validation Accuracy,74.15
Validation Loss,0.82139


In [ ]:
# Sweep configuration dictionary
sweep_config = {
    'method': 'bayes',  # 'bayes' will use Bayesian optimization to find optimal hyperparameters, other options are 'grid' or 'random'

    'metric': {
        'name': 'Validation Accuracy',  # Metric to optimize during the sweep
        'goal': 'maximize'  # Goal is to maximize the validation accuracy
    },

    'parameters': {

        'freeze_percent': {
            'values': [0.75]  # Different percentages of layers to freeze during fine-tuning
        },

        'model_name': {
            'values': ['resnet']  # Pre-trained model options for fine-tuning
        },

        'freeze_all_except_last_layer': {
            'values': ['Yes']  # Option to freeze all layers except the last one or not
        },

        'epochs': {
            'value': 6  # Number of epochs for each training run
        }
    }
}

# Initialize the W&B sweep using the config
sweep_id = wandb.sweep(sweep_config, project="DA6401_A-2_Part-B")  # Initialize sweep in the project "DA6401_A-2_Part-B"


Create sweep with ID: m3hs7ere
Sweep URL: https://wandb.ai/ma23m013-iit-madras/DA6401_A-2_Part-B/sweeps/m3hs7ere


In [ ]:
def main():
    '''
    WandB calls the main function each time with a different combination of hyperparameters.
    We can retrieve the configuration values from `wandb.config` and use them in our model training.

    This function initializes the W&B run, loads the pre-trained model with specified hyperparameters,
    prepares the dataset, and starts the training process.
    '''

    # Initialize W&B run
    with wandb.init() as run:
        # Define the run name based on hyperparameters for better tracking
        run_name = "ep" + str(wandb.config.epochs) + "_fp-" + str(wandb.config.freeze_percent) + "_model-" + str(wandb.config.model_name)
        wandb.run.name = run_name  # Set the name for this run in W&B

        # Load the pre-trained model based on configuration
        model = pretrain_model(
            model_name=wandb.config.model_name,
            freeze_percent=wandb.config.freeze_percent,
            freeze_all_except_last_layer=wandb.config.freeze_all_except_last_layer
        )

        # Move the model to the appropriate device (CPU or GPU)
        model = model.to(device)

        # Define the directory where the training data is located
        data_dir = '/kaggle/input/inaturalist-dataset/inaturalist_12K/train'

        # Load the training and validation data with or without data augmentation
        train_data, val_data = load_dataset(data_dir, apply_augmentation='No')

        # Train the model with the loaded data and configurations
        train_and_validate_model(model, train_data, val_data, num_epochs=wandb.config.epochs)


# Start the W&B sweep, calling the main function for the specified number of times
wandb.agent(sweep_id, function=main, count=10)  # Number of runs = 10 in this case, but can be increased
wandb.finish()  # Finish the W&B run after the sweep is complete


wandb: Agent Starting Run: qskphf4h with config:
wandb: 	epochs: 6
wandb: 	freeze_all_except_last_layer: Yes
wandb: 	freeze_percent: 0.75
wandb: 	model_name: resnet
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Epoch 1/6, Train Loss: 1.1018, Train Accuracy: 64.23%
Epoch 1/6, Validation Loss: 0.8264, Validation Accuracy: 73.05%
Epoch 2/6, Train Loss: 0.8378, Train Accuracy: 72.00%
Epoch 2/6, Validation Loss: 0.8181, Validation Accuracy: 72.45%
Epoch 3/6, Train Loss: 0.7838, Train Accuracy: 74.15%
Epoch 3/6, Validation Loss: 0.8135, Validation Accuracy: 74.05%
Epoch 4/6, Train Loss: 0.7586, Train Accuracy: 74.58%
Epoch 4/6, Validation Loss: 0.8641, Validation Accuracy: 72.25%
Epoch 5/6, Train Loss: 0.7334, Train Accuracy: 75.71%
Epoch 5/6, Validation Loss: 0.8219, Validation Accuracy: 74.10%
Epoch 6/6, Train Loss: 0.7109, Train Accuracy: 76.31%
Epoch 6/6, Validation Loss: 0.8885, Validation Accuracy: 71.70%
Training complete!


Epoch,▁▂▄▅▇█
Train Accuracy,▁▅▇▇██
Train Loss,█▃▂▂▁▁
Validation Accuracy,▅▃█▃█▁
Validation Loss,▂▁▁▆▂█
Epoch,6
Train Accuracy,76.30954
Train Loss,0.7109
Validation Accuracy,71.7
Validation Loss,0.88853


wandb: Agent Starting Run: gupim4nc with config:
wandb: 	epochs: 6
wandb: 	freeze_all_except_last_layer: Yes
wandb: 	freeze_percent: 0.75
wandb: 	model_name: resnet


Epoch 1/6, Train Loss: 1.1343, Train Accuracy: 64.11%
Epoch 1/6, Validation Loss: 0.8806, Validation Accuracy: 70.55%
Epoch 2/6, Train Loss: 0.8407, Train Accuracy: 72.77%
Epoch 2/6, Validation Loss: 0.7911, Validation Accuracy: 74.00%
Epoch 3/6, Train Loss: 0.7857, Train Accuracy: 73.93%
Epoch 3/6, Validation Loss: 0.8454, Validation Accuracy: 73.20%
Epoch 4/6, Train Loss: 0.7529, Train Accuracy: 75.06%
Epoch 4/6, Validation Loss: 0.8066, Validation Accuracy: 74.30%
Epoch 5/6, Train Loss: 0.7362, Train Accuracy: 75.66%
Epoch 6/6, Train Loss: 0.7196, Train Accuracy: 76.13%
Epoch 6/6, Validation Loss: 0.8141, Validation Accuracy: 74.30%
Training complete!


...

Epoch 1/6, Train Loss: 1.1064, Train Accuracy: 64.63%
Epoch 1/6, Validation Loss: 0.8939, Validation Accuracy: 71.05%
Epoch 2/6, Train Loss: 0.8450, Train Accuracy: 71.96%
Epoch 2/6, Validation Loss: 0.8141, Validation Accuracy: 73.10%
Epoch 3/6, Train Loss: 0.7840, Train Accuracy: 73.87%
Epoch 3/6, Validation Loss: 0.8426, Validation Accuracy: 72.40%
Epoch 4/6, Train Loss: 0.7627, Train Accuracy: 74.72%
Epoch 4/6, Validation Loss: 0.7997, Validation Accuracy: 74.55%
Epoch 5/6, Train Loss: 0.7442, Train Accuracy: 75.05%
Epoch 5/6, Validation Loss: 0.8267, Validation Accuracy: 73.10%
Epoch 6/6, Train Loss: 0.7175, Train Accuracy: 76.31%
Epoch 6/6, Validation Loss: 0.8411, Validation Accuracy: 72.95%
Training complete!


Epoch,▁▂▄▅▇█
Train Accuracy,▁▅▇▇▇█
Train Loss,█▃▂▂▁▁
Validation Accuracy,▁▅▄█▅▅
Validation Loss,█▂▄▁▃▄
Epoch,6
Train Accuracy,76.30954
Train Loss,0.71746
Validation Accuracy,72.95
Validation Loss,0.84111


wandb: Agent Starting Run: uxdekj2k with config:
wandb: 	epochs: 6
wandb: 	freeze_all_except_last_layer: Yes
wandb: 	freeze_percent: 0.75
wandb: 	model_name: resnet


Epoch 1/6, Train Loss: 1.1108, Train Accuracy: 64.43%
Epoch 1/6, Validation Loss: 0.8742, Validation Accuracy: 70.65%
Epoch 2/6, Train Loss: 0.8332, Train Accuracy: 72.26%
Epoch 2/6, Validation Loss: 0.8329, Validation Accuracy: 72.80%
Epoch 3/6, Train Loss: 0.7780, Train Accuracy: 73.66%
Epoch 3/6, Validation Loss: 0.8128, Validation Accuracy: 73.45%
Epoch 4/6, Train Loss: 0.7447, Train Accuracy: 75.56%
Epoch 4/6, Validation Loss: 0.8646, Validation Accuracy: 72.55%
Epoch 5/6, Train Loss: 0.7258, Train Accuracy: 76.18%
Epoch 5/6, Validation Loss: 0.8189, Validation Accuracy: 73.75%
Epoch 6/6, Train Loss: 0.7252, Train Accuracy: 75.96%
Epoch 6/6, Validation Loss: 0.7993, Validation Accuracy: 74.85%
Training complete!


Epoch,▁▂▄▅▇█
Train Accuracy,▁▆▆███
Train Loss,█▃▂▁▁▁
Validation Accuracy,▁▅▆▄▆█
Validation Loss,█▄▂▇▃▁
Epoch,6
Train Accuracy,75.95949
Train Loss,0.72522
Validation Accuracy,74.85
Validation Loss,0.79928
